In [1]:
import pyedflib
import numpy as np
import xmltodict
import json
import mne
import matplotlib
import math
import pathlib
from mne_extras import write_mne_edf
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side

In [2]:
matplotlib.use('Qt5Agg')

In [48]:
def analyze(this_epochs):
    normal = this_epochs.copy().pick_channels(['Fp1.'])
    ear_ref = this_epochs.copy().set_eeg_reference(['T7..']).pick_channels(['Fp1.'])
    head_ref = this_epochs.copy().set_eeg_reference(['Fp2.']).pick_channels(['Fp1.'])
#     ear_ref.plot()
#     head_ref.plot()
    
    power = []
    delta = []
    theta = []
    alpha = []
    beta = []
    gamma = []

#     for epochs in this_epochs:  

#         variances.append(np.var(out_signal))
    epochs = this_epochs
    psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs, fmin=0.5, fmax=44.5, tmin=None, tmax=None)
    total_sum_pds = np.sum(psds_total)

    psds_power, frqs_power = mne.time_frequency.psd_multitaper(epochs, fmin=45, fmax=55, tmin=None, tmax=None)
    power_sum_pds = np.sum(psds_power)
    power_ratio = (power_sum_pds) / (total_sum_pds)
    power.append(power_ratio)

    psds_delta, frqs_delta = mne.time_frequency.psd_multitaper(epochs, fmin=1.5, fmax=4.5, tmin=None, tmax=None)
    delta_sum_pds = np.sum(psds_delta)
    delta_ratio = (delta_sum_pds) / (total_sum_pds)
    delta.append(delta_ratio)

    psds_theta, frqs_theta = mne.time_frequency.psd_multitaper(epochs, fmin=4.5, fmax=7.5, tmin=None, tmax=None)
    theta_sum_pds = np.sum(psds_theta)
    theta_ratio = (theta_sum_pds) / (total_sum_pds)
    theta.append(theta_ratio)

    psds_alpha, frqs_alpha = mne.time_frequency.psd_multitaper(epochs, fmin=7.5, fmax=12.5, tmin=None, tmax=None)
    alpha_sum_pds = np.sum(psds_alpha)
    alpha_ratio = (alpha_sum_pds) / (total_sum_pds)
    alpha.append(alpha_ratio)

    psds_beta, frqs_beta = mne.time_frequency.psd_multitaper(epochs, fmin=14.5, fmax=29.5, tmin=None, tmax=None)
    beta_sum_pds = np.sum(psds_beta)
    beta_ratio = (beta_sum_pds) / (total_sum_pds)
    beta.append(beta_ratio)

    psds_gamma, frqs_gamma = mne.time_frequency.psd_multitaper(epochs, fmin=29.5, fmax=45.5, tmin=None, tmax=None)
    gamma_sum_pds = np.sum(psds_gamma)
    gamma_ratio = (gamma_sum_pds) / (total_sum_pds)
    gamma.append(gamma_ratio)
    
    df = this_epochs.to_data_frame()
    var = np.var(df)
#     print((var))
#     print(len(var))
#     print(sum(df.var()[2:]))
    var_avg = sum(df.var()[2:]) / 64
    result = [power, delta, theta, alpha, beta, gamma, [var_avg]]
#     print(result[6])
    return result


In [ ]:
exp = []
expRaw = []
event_dict = {'divide':1}
n = 109
epochs = []
variances = []

headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch3', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch4', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch5', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch6', 
            'dimension': 'uV',
            'sample_rate': 160,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]

wb = Workbook()
ws = wb.active
ws.title = 'EEG Compositional Analysis'
ws.sheet_properties.tabColor = "1072BA"

ws.cell(row=1, column=1).value = 'normal.power'
ws.cell(row=1, column=2).value = 'normal.delta'
ws.cell(row=1, column=3).value = 'normal.theta'
ws.cell(row=1, column=4).value = 'normal.alpha'
ws.cell(row=1, column=5).value = 'normal.beta'
ws.cell(row=1, column=6).value = 'normal.gamma'
ws.cell(row=1, column=7).value = 'normal.var'

# ws.cell(row=1, column=7).value = 'ear.power'
# ws.cell(row=1, column=8).value = 'ear.delta'
# ws.cell(row=1, column=9).value = 'ear.theta'
# ws.cell(row=1, column=10).value = 'ear.alpha'
# ws.cell(row=1, column=11).value = 'ear.beta'
# ws.cell(row=1, column=12).value = 'ear.gamma'

# ws.cell(row=1, column=13).value = 'forehead.power'
# ws.cell(row=1, column=14).value = 'forehead.delta'
# ws.cell(row=1, column=15).value = 'forehead.theta'
# ws.cell(row=1, column=16).value = 'forehead.alpha'
# ws.cell(row=1, column=17).value = 'forehead.beta'
# ws.cell(row=1, column=18).value = 'forehead.gamma'

for i in range(1,8):
    ws.cell(row=1, column=i).fill = PatternFill(fill_type='solid',
                start_color='00CCFFFF',
                end_color='FF000000')
ws.sheet_format.baseColWidth=15
    
for i in range(0,n):
    print('this is: ' + str(i))
    exp.append(mne.datasets.eegbci.load_data(i+1, 2, path=None, force_update=False, update_path=None, base_url='https://physionet.org/files/eegmmidb/1.0.0/', verbose=None)[0])
    expRaw.append(mne.io.read_raw_edf(exp[i]))
    
    
    
    events_all = mne.events_from_annotations(expRaw[i])
    new_events = mne.make_fixed_length_events(expRaw[i], duration=2.)
    events = events_all[0]
    event_id = events_all[1]
    # two on top of forehead and one on top of left ear
    this_epochs = mne.Epochs(expRaw[i], new_events, reject_by_annotation=False, preload=True)
    epochs.append(this_epochs)
    
#     df = this_epochs.to_data_frame()

#     #original data for forehead and ear eeg (default refrence at Cz)
#     out_raw_one = df['Fp1.']
#     out_raw_two = df['Fp2.']
#     out_raw_three = df['T7..']
    
#     #forehead eeg with ear as refrence
#     out_raw_four = df['Fp1.']  - df['T7..']
#     out_raw_five = df['Fp2.']  - df['T7..']
    
#     #one forehead eeg with the other as refrence
#     out_raw_six = df['Fp1.']  - df['Fp2.']

#     out_signal = [np.array(out_raw_one), np.array(out_raw_two), np.array(out_raw_three), np.array(out_raw_four), np.array(out_raw_five), np.array(out_raw_six)]
    
#     with open('./mne_data_analysis_output/forthCH_Fp1_Fp2'+str(i)+'.edf', 'w') as output:
#         flag = pyedflib.highlevel.write_edf(output.name, out_signal, headers, header=None, digital=False, file_type=-1, block_size=1)
#         #print(flag)
        
    result = analyze(this_epochs)
    print(len(result))
    print(len(result[0]))
    for x in range(0,len(result)):
        ws.cell(row=i+2, column=x+1).value = result[x][0]
#     for x in range(0,len(result)):
#         ws.cell(row=i+2, column=len(result)+x+1).value = result[x][0]
#     for x in range(0,len(result)):
#         ws.cell(row=i+2, column=2*len(result)+x+1).value = result[x][0]
    
wb.save('mne_data_analysis.xlsx')
        
    


    


In [44]:
print(['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'])


['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']


In [6]:
mne.datasets.brainstorm.bst_resting.data_path(path=None, force_update=False, update_path=True, download=True, accept=False, verbose=None)

'C:\\Users\\admin\\mne_data\\MNE-brainstorm-data\\bst_resting'